## Hyper Parameter Tuning

In [21]:
# Import Libraries
import pandas as pd
import tensorflow as tf
import pickle

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline

from scikeras.wrappers import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [22]:
# Read the dataset
df = pd.read_csv("../dataset/Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### Data Preprocessing

In [23]:
# Drop the columns that are not needed
df = df.drop(["RowNumber", "CustomerId", "Surname"], axis=1)

# Encode the categorical variables
label_encoder = LabelEncoder()
df["Gender"] = label_encoder.fit_transform(df["Gender"])

# One-hot encode the 'Geography' column
ohe_geo = OneHotEncoder(handle_unknown="ignore")
geo_encoded = ohe_geo.fit_transform(df[["Geography"]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=ohe_geo.get_feature_names_out(["Geography"]))

# Combine the encoded columns with the original dataframe
df = pd.concat([df.drop("Geography", axis=1), geo_encoded_df], axis=1)

In [ ]:
# Split the dataset into features and target variable
X = df.drop("Exited", axis=1)
y = df["Exited"]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [25]:
# Save the label encoder and one-hot encoder
pkl_files = ["label_encoder.pkl", "ohe_geo.pkl", "scaler.pkl"]

for pkl in pkl_files:
  with open(pkl, "wb") as file:
    if pkl == "label_encoder.pkl":
      pickle.dump(label_encoder, file)
    elif pkl == "ohe_geo.pkl":
      pickle.dump(ohe_geo, file)
    else:
      pickle.dump(scaler, file)

In [26]:
# Define function to create the model and try different hyperparameters
def create_model(neurons=32, layers=1):
  model = Sequential() # Initialize the model
  model.add(Dense(neurons, activation="relu", input_shape=(X_train.shape[1],))) # Input layer with specified number of neurons
  
  # Check if more layers are needed
  for _ in range(layers - 1):
    model.add(Dense(neurons, activation="relu"))
  
  model.add(Dense(1, activation="sigmoid"))  # Output layer for classification
  model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
  
  return model

In [27]:
# Create a KerasClassifier
model = KerasClassifier(layers=1, neurons=32 ,build_fn=create_model, verbose=1)

In [28]:
# Define the grid search parameters
param_grid = {
  'neurons': [16, 32, 64, 128],
  'layers': [1, 2],
  'epochs': [50, 100],
}

# Perform grid search with cross-validation
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=1)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters and score
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

Fitting 3 folds for each of 16 candidates, totalling 48 fits


ValueError: 
All the 48 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
48 fits failed with the following error:
Traceback (most recent call last):
  File "c:\MySkill\Data Science\Projects\ann-dl-project\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\MySkill\Data Science\Projects\ann-dl-project\venv\lib\site-packages\scikeras\wrappers.py", line 1501, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "c:\MySkill\Data Science\Projects\ann-dl-project\venv\lib\site-packages\scikeras\wrappers.py", line 770, in fit
    self._fit(
  File "c:\MySkill\Data Science\Projects\ann-dl-project\venv\lib\site-packages\scikeras\wrappers.py", line 925, in _fit
    X, y =